In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

In [2]:
data = pd.read_csv("C:\\Users\\melchiz shroff\\Downloads\\bbc-text\\bbc-text.csv")

In [3]:
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [4]:
data['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [5]:
data.shape

(2225, 2)

In [67]:
train_size = int(len(data)*0.8)
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [68]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [71]:
max_words = 1000
tokenize = tf.keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [75]:
from sklearn.preprocessing import LabelEncoder

In [76]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [77]:
#One hot encoding sparse matrix
y_train = tf.keras.utils.to_categorical(y_train,5)
y_test= tf.keras.utils.to_categorical(y_test,5)

In [82]:
model= tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(512,input_shape=(1000,)))
model.add(tf.keras.layers.Activation('relu'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [83]:
model.add(tf.keras.layers.Dense(5))
model.add(tf.keras.layers.Activation('softmax'))

In [84]:
#Compiling the model
model.compile(loss='categorical_crossentropy',
             optimizer= 'adam',
             metrics= ['accuracy'])

In [87]:
history = model.fit(x_train, y_train,
                    batch_size=16,
                    epochs=4,
                    verbose=1,
                    validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/4
1602/1602 [==============================] - 1s 315us/sample - loss: 0.0040 - acc: 1.0000 - val_loss: 0.1144 - val_acc: 0.9607
Epoch 2/4
1602/1602 [==============================] - 1s 327us/sample - loss: 0.0019 - acc: 1.0000 - val_loss: 0.1144 - val_acc: 0.9607
Epoch 3/4
1602/1602 [==============================] - 1s 322us/sample - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1186 - val_acc: 0.9607
Epoch 4/4
1602/1602 [==============================] - 1s 322us/sample - loss: 7.5503e-04 - acc: 1.0000 - val_loss: 0.1171 - val_acc: 0.9607


In [90]:
#Evaluating the model
score = model.evaluate(x_test, y_test,
                       batch_size=16, verbose=1)

445/445 [==============================] - 0s 85us/sample - loss: 0.1157 - acc: 0.9596


In [92]:
y_pred = model.predict(x_test)

In [94]:
y_test_label = []
y_pred_label = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_label.append(one_hot_index)

for i in range(0, len(y_pred)):
    probs = y_pred[i]
    predicted_index = np.argmax(probs)
    y_pred_label.append(predicted_index)

In [95]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_label, y_pred_label)
cm

array([[99,  3,  3,  0,  1],
       [ 1, 61,  1,  0,  3],
       [ 2,  0, 83,  0,  1],
       [ 0,  2,  0, 99,  0],
       [ 1,  0,  0,  0, 85]], dtype=int64)